In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.utils.globus_utils import GlobusConfig, create_globus_tc
from multiplex_pipeline.utils.file_utils import globus_dir_exists, GlobusPathConverter
from multiplex_pipeline.core_cutting.file_io import GlobusFileStrategy

In [2]:
# experiment settings file path
settings_path = r'C:\BLCA-1_Analysis_todel\analysis_settings_BLCA1.yaml'

# globus config
config_path = Path(r'D:\globus_config\globus_config.yaml')


In [3]:
# load analysis configuration
settings = load_analysis_settings(settings_path)

In [4]:
settings

{'image_dir': '/CellDive/BLCA-1/BLCA-1_Final',
 'analysis_name': 'BLCA-1_Analysis_todel',
 'local_analysis_dir': 'C:/',
 'remote_analysis_dir': '/ix1/kkedziora/blca_analysis',
 'log_dir': WindowsPath('C:/BLCA-1_Analysis_todel/logs'),
 'detection_image': 'BLCA-1_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_file_path': WindowsPath('C:/BLCA-1_Analysis_todel/cores.csv'),
 'cores_dir_tif': WindowsPath('C:/BLCA-1_Analysis_todel/temp'),
 'cores_dir_output': WindowsPath('C:/BLCA-1_Analysis_todel/cores'),
 'include_channels': None,
 'exclude_channels': ['008_ECad'],
 'use_markers': ['DAPI'],
 'ignore_markers': ['Antibody1',
  'TNFa',
  'Snail1',
  'SKP2',
  'ProgRc',
  'Plk1',
  'PH3',
  'PDL1',
  'p65',
  'p130',
  'p-p130',
  'p-Cdc6',
  'LAG3',
  'IL-8',
  'HER2',
  'ERa',
  'EpCAM',
  'E2F1',
  'cycD3',
  'cycB2',
  'CDC25C',
  'CD86',
  'CD73',
  'CD69',
  'CD62L',
  'CD56',
  'CD4',
  'CD25',
  'CD19',
  'CD27',
  'CCR7',
  'cCASP3'],
 'segmentation': {'package': 'instaseg',
  'model': 

In [ ]:
# get Globus settings
gc = GlobusConfig.from_config_files(config_path, from_collection = 'cbi_collection_id', to_collection = 'crcd_collection_id')
tc = create_globus_tc(gc.client_id, gc.transfer_tokens)

In [6]:
# check if the destination analysis directory exists 
remote_analysis_dir = settings["remote_analysis_dir"] + '/' + settings["analysis_name"]

globus_dir_exists(tc, gc.destination_collection_id, remote_analysis_dir)

True

In [7]:
# build a transfer map

# if transfering from Windows with multiple drives exposed in Globus use "multi_drive"
# if transfering from Windows with a single drive exposed in Globus 
# or if transfering from CIFs drive use "single_drive"
conv = GlobusPathConverter(layout="multi_drive")

local_settings_path = conv.windows_to_globus(settings_path)
remote_settings_path = remote_analysis_dir + '/' + Path(settings_path).name
    
local_cores_info_path = conv.windows_to_globus(settings['core_info_file_path'].with_suffix('.pkl'))
remote_cores_info_path = remote_analysis_dir + '/' + Path(local_cores_info_path).name

remote_cores_info_path = remote_analysis_dir + '/' + Path(local_cores_info_path).name

transfer_map = {}
transfer_map['settings'] = (local_settings_path, remote_settings_path)
transfer_map['core_definitions'] = (local_cores_info_path, remote_cores_info_path)
transfer_map

{'settings': ('/C/BLCA-1_Analysis_todel/analysis_settings_BLCA1.yaml',
  '/ix1/kkedziora/blca_analysis/BLCA-1_Analysis_todel/analysis_settings_BLCA1.yaml'),
 'core_definitions': ('/C/BLCA-1_Analysis_todel/cores.pkl',
  '/ix1/kkedziora/blca_analysis/BLCA-1_Analysis_todel/cores.pkl')}

In [8]:
# submit transfer
transfer = GlobusFileStrategy(tc=tc, transfer_map=transfer_map, gc=gc, cleanup_enabled = False)

2025-10-07 14:06:14.577 | INFO     | multiplex_pipeline.core_cutting.file_io:submit_all_transfers:74 - Submitted transfer for settings to /ix1/kkedziora/blca_analysis/BLCA-1_Analysis_todel/analysis_settings_BLCA1.yaml (task_id=4f631e56-a3a8-11f0-aecb-0affca67c55f)
2025-10-07 14:06:15.053 | INFO     | multiplex_pipeline.core_cutting.file_io:submit_all_transfers:74 - Submitted transfer for core_definitions to /ix1/kkedziora/blca_analysis/BLCA-1_Analysis_todel/cores.pkl (task_id=4fd6be78-a3a8-11f0-b9df-0affca67c55f)


In [10]:
# monitor transfer
print(f'Status of {transfer.pending[0][1]}: {tc.get_task(transfer.pending[0][0])["status"]}')
print(f'Status of {transfer.pending[1][1]}: {tc.get_task(transfer.pending[1][0])["status"]}')

Status of /ix1/kkedziora/blca_analysis/BLCA-1_Analysis_todel/analysis_settings_BLCA1.yaml: SUCCEEDED
Status of /ix1/kkedziora/blca_analysis/BLCA-1_Analysis_todel/cores.pkl: SUCCEEDED
